In [1]:
import os
%pwd

'e:\\End-to-End-Project\\Data-Science-first\\datascienceproject\\research'

In [2]:
os.chdir('../')
%pwd

'e:\\End-to-End-Project\\Data-Science-first\\datascienceproject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
from dotenv import load_dotenv

load_dotenv()
os.environ["MLFLOW_TRACKING_URI"]= os.getenv('MLFLOW_URI')
os.environ["MLFLOW_TRACKING_USERNAME"]=os.getenv('NAME')
os.environ["MLFLOW_TRACKING_PASSWORD"]= os.getenv('SECRET_ACCESS_KEY')

In [14]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    model_metrics_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [15]:
from src.datascience.constant import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManger:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            model_metrics_name= config.model_metrics_name,
            all_params = params,
            target_column= schema.name,
            mlflow_uri = 'https://dagshub.com/Bilal-ahmad8/datascienceproject.mlflow'
        )
        return model_evaluation_config


In [17]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import mlflow.sklearn
import numpy as np
from urllib.parse import urlparse
from src.datascience import logger

In [18]:
class ModelEvaluation:
    def __init__(self, config= ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2Score = r2_score(actual,pred)
        return rmse, mae, r2Score
    

    def log_into_mlflow(self):
        data = pd.read_csv(self.config.test_data_path)

        test_x = data.drop([self.config.target_column], axis=1)
        test_y = data[[self.config.target_column]]

        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        #tracking_uri_type = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            pred = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, pred)

            scores = {'rmse': rmse, "mae": mae, 'r2_score': r2}

            save_json(path=Path(self.config.model_metrics_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            mlflow.sklearn.log_model(model,"model")



        

In [19]:
try:
    config = ConfigurationManger()
    evaluate_config = config.get_model_evaluation_config()
    evaluate = ModelEvaluation(evaluate_config)
    evaluate.log_into_mlflow()

except Exception as e:
    raise e

[ 2025-02-26 16:02:39,352 : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ 2025-02-26 16:02:39,355 : INFO : common : yaml file: params.yaml loaded successfully ]
[ 2025-02-26 16:02:39,359 : INFO : common : yaml file: schema.yaml loaded successfully ]
[ 2025-02-26 16:02:39,360 : INFO : common : created directory at: artifacts ]
[ 2025-02-26 16:02:39,363 : INFO : common : created directory at: artifacts/model_evaluation ]
[ 2025-02-26 16:02:40,442 : INFO : common : json file saved at: artifacts\model_evaluation\metrics.json ]


2025/02/26 16:02:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run wise-swan-698 at: https://dagshub.com/Bilal-ahmad8/datascienceproject.mlflow/#/experiments/0/runs/1704f7ec3a5c49eb8ae3ab10370d100b
🧪 View experiment at: https://dagshub.com/Bilal-ahmad8/datascienceproject.mlflow/#/experiments/0
